In [1]:
import json, time
import pyautogui
import socket, uuid, time
import mss
import mss.tools
import tensorflow as tf
from PIL import Image, ImageDraw
import cv2
import numpy as np
import sys, os, decimal, json
sys.path.append(os.path.realpath('..'))
from MiningBot.EveInterface.Interface import Interface, get_cells, get_row_points, get_col_points

ModuleNotFoundError: No module named 'pyautogui'

In [ ]:
config_dir = r'../MiningBot/Configs/configs.json'
config = json.load(open(config_dir))[socket.gethostname()]
game = Interface(config_dir=config_dir)

In [2]:
classifiers = {}

for clsf_name in config['Classifiers'].keys():
    f = open(config['Classifiers'][clsf_name]['class_location'], "r")
    classifiers[clsf_name] = {
        'model': tf.keras.models.load_model(config['Classifiers'][clsf_name]['model_location']),
        'classes': json.loads(f.read()),
        'image_resize': tuple(config['Classifiers'][clsf_name]['image_resize']),
        'save_images': bool(config['Classifiers'][clsf_name]['save_images']),
    }

NameError: name 'config' is not defined

In [3]:
def get_screen():
    with mss.mss() as sct:
        mon = sct.monitors[config['monitor_number']]

        # The screen part to capture
        monitor = {
            "top": mon["top"],
            "left": mon["left"],
            "width": mon["width"],
            "height": mon["height"],
            "mon": config['monitor_number'],
        }

        # Grab the data
        img = np.array(sct.grab(monitor))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        return img

In [46]:
def execute_clsf(img, clsf_name):
    img = img.resize(
        (classifiers[clsf_name]['image_resize'][1], classifiers[clsf_name]['image_resize'][0]),
        # TF trains backwards
        resample=Image.Resampling.NEAREST)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    predictions = classifiers[clsf_name]['model'].predict(img_array)
    scores = tf.nn.softmax(predictions[0])
    result = {
        'argmax_index': np.argmax(scores),
        'value_at_argmax': scores[np.argmax(scores)].numpy(),
        'pass_general_tollerance': scores[np.argmax(scores)].numpy() > 0.5,
        'class': classifiers[clsf_name]['classes'][np.argmax(scores)],
        'classes': classifiers[clsf_name]['classes'],
        'scores': scores.numpy().tolist()
    }
    return result

In [47]:
def get_processed_cords(x, y):
    return x + config['monitor_offset_x'], y + config['monitor_offset_y']

In [41]:
while True:
    while True:
        send_home = False
        
        pyautogui.moveTo(get_processed_cords(1283, 133))
        time.sleep(0.1)
        pyautogui.click(button='left')
        time.sleep(2)       
        search_df = game.get_search_data(rows=13, refresh_screen=True)
        search_df = search_df[(~search_df['Name'].str.contains('Jita')) & (~search_df['Name'].str.contains('Amsen'))].reset_index()
        del search_df['index']
        click_target = search_df.loc[0, 'click_target']
        pyautogui.moveTo(click_target)
        time.sleep(0.1)
        pyautogui.click(button='right')
        time.sleep(0.1)
        box_top_left = (935,204)
        box_bottom_right = (1200,600)
        img = get_screen()
        img = img.crop([box_top_left[0], box_top_left[1], box_bottom_right[0], box_bottom_right[1]])
        img.save(f'O:\\source\\repos\\EVE-Online-Bot\\training_data\\unclass\\{uuid.uuid1()}.png')
        
        dest_result = execute_clsf(img, 'set_dest')
        print(dest_result)
        
        if dest_result['class'] == 'second_pos':
            pyautogui.moveTo(click_target[0] + 50, click_target[1] + 60)
        elif dest_result['class'] == 'seventh_pos':
            pyautogui.moveTo(click_target[0] + 50, click_target[1] + 234)
        elif dest_result['class'] == 'third_pos':
            pyautogui.moveTo(click_target[0] + 50, click_target[1] + 109)

        time.sleep(0.1)
        pyautogui.click(button='left')
        time.sleep(1)

        nav_point_xy = get_processed_cords(137, 378)
        while True:
            pyautogui.moveTo(nav_point_xy)
            time.sleep(0.1)
            pyautogui.click(button='right')
            time.sleep(0.1)
            box = (102, 289, 458, 658)
            img = get_screen()
            state_result = execute_clsf(img, 'game_state')
            print(state_result)
            img = img.crop(box)
            print('saving...')
            img.save(f'O:\\source\\repos\\EVE-Online-Bot\\training_data\\unclass\\{uuid.uuid1()}.png')
            nav_result = execute_clsf(img, 'nav_options')
            print(nav_result)
            if state_result['class'] == 'in_flight':
                if nav_result['class'] == 'dock_now':
                    print('Docking Now...')
                    pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 75)
                    time.sleep(0.1)
                    pyautogui.click(button='left')
                elif nav_result['class'] == 'jump_though_first':
                    print('jumping fist pos...')
                    pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 25)
                    time.sleep(0.1)
                    pyautogui.click(button='left')
                elif nav_result['class'] == 'jump_through_second':
                    print('jumping second pos...')
                    pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 50)
                    time.sleep(0.1)
                    pyautogui.click(button='left')
                elif nav_result['class'] == 'warp_to_dock':
                    print('Warp To Dock pos...')
                    pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 75)
                    time.sleep(0.1)
                    pyautogui.click(button='left')
                else:
                    print('do nothing nav...')
            elif state_result['class'] == 'in_hanger':
                time.sleep(2)

                pyautogui.moveTo(get_processed_cords(*config['hanger_target']))  # ensure we have mining hold selected
                time.sleep(0.1)
                pyautogui.click(button='left')
                time.sleep(1)


                pyautogui.moveTo(get_processed_cords(*config['click_and_drag_inv_box'][2:4]))  #
                time.sleep(0.1)
                x, y = get_processed_cords(*config['click_and_drag_inv_box'][0:2])
                pyautogui.dragTo(x, y, 1, button='left')
                time.sleep(0.1)
                pyautogui.moveTo(get_processed_cords(*config['Load_to_mininghold_click_and_drag_inv_line'][0:2]))
                time.sleep(0.1)
                x, y = get_processed_cords(*config['Load_to_mininghold_click_and_drag_inv_line'][2:4])
                pyautogui.dragTo(x, y, 1, button='left')


                time.sleep(2)

                img = get_screen()
                menu_result = execute_clsf(img, 'hanger_menus')
                print(menu_result)
                if menu_result['class'] == 'set_quant':
                    pyautogui.moveTo(get_processed_cords(*config['set_quant_target']))  # ensure we have mining hold selected
                    time.sleep(0.1)
                    pyautogui.click(button='left')
                    time.sleep(1)
                    send_home = True

                time.sleep(0.1)
                pyautogui.moveTo(get_processed_cords(*config['exit_hanger_target']))
                time.sleep(0.1)
                pyautogui.click(button='left')
                time.sleep(30)
                break
            else:
                print('do nothing state...')
            time.sleep(0.1)
            pyautogui.moveTo(get_processed_cords(100, 100))
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(10)
        if send_home:
            break


    search_df = game.get_search_data(rows=13, refresh_screen=True)
    search_df = search_df[search_df['Name'].str.contains('Jita')].reset_index()
    del search_df['index']
    click_target = search_df.loc[0, 'click_target']
    pyautogui.moveTo(click_target)
    time.sleep(0.1)
    pyautogui.click(button='right')
    time.sleep(0.1)
    pyautogui.moveTo(click_target[0] + 50, click_target[1] + 60)
    time.sleep(0.1)
    pyautogui.click(button='left')
    time.sleep(1)         

    nav_point_xy = get_processed_cords(137, 378)
    while True:
        pyautogui.moveTo(nav_point_xy)
        time.sleep(0.1)
        pyautogui.click(button='right')
        time.sleep(0.1)
        box = (102, 289, 458, 658)
        img = get_screen()
        state_result = execute_clsf(img, 'game_state')
        print(state_result)
        img = img.crop(box)
        nav_result = execute_clsf(img, 'nav_options')
        img.save(f'O:\\source\\repos\\EVE-Online-Bot\\training_data\\unclass\\{uuid.uuid1()}.png')
        print(nav_result)
        if state_result['class'] == 'in_flight':
            if nav_result['class'] == 'dock_now':
                print('Docking Now...')
                pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 75)
                time.sleep(0.1)
                pyautogui.click(button='left')
            elif nav_result['class'] == 'jump_though_first':
                print('jumping fist pos...')
                pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 25)
                time.sleep(0.1)
                pyautogui.click(button='left')
            elif nav_result['class'] == 'jump_through_second':
                print('jumping second pos...')
                pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 50)
                time.sleep(0.1)
                pyautogui.click(button='left')
            elif nav_result['class'] == 'warp_to_dock':
                print('Warp To Dock pos...')
                pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 75)
                time.sleep(0.1)
                pyautogui.click(button='left')
            else:
                print('do nothing nav...')
        elif state_result['class'] == 'in_hanger':
            time.sleep(2)

            pyautogui.moveTo(get_processed_cords(*config['mining_hold_target']))  # ensure we have mining hold selected
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(1)


            pyautogui.moveTo(get_processed_cords(*config['click_and_drag_inv_box'][2:4]))  #
            time.sleep(0.1)
            x, y = get_processed_cords(*config['click_and_drag_inv_box'][0:2])
            pyautogui.dragTo(x, y, 1, button='left')
            time.sleep(0.1)
            pyautogui.moveTo(get_processed_cords(*config['Unload_nav_bot_click_and_drag_inv_line'][0:2]))
            time.sleep(0.1)
            x, y = get_processed_cords(*config['Unload_nav_bot_click_and_drag_inv_line'][2:4])
            pyautogui.dragTo(x, y, 1, button='left')
            time.sleep(0.1)
            pyautogui.moveTo(get_processed_cords(*config['exit_hanger_target']))
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(30)
            break
        else:
            print('do nothing state...')
        time.sleep(0.1)
        pyautogui.moveTo(get_processed_cords(100, 100))
        time.sleep(0.1)
        pyautogui.click(button='left')
        time.sleep(10)

1/1 [==============================] - 0s 47ms/step
{'argmax_index': 0, 'value_at_argmax': 0.9929731, 'pass_general_tollerance': True, 'class': 'second_pos', 'classes': ['second_pos', 'seventh_pos', 'third_pos'], 'scores': [0.9929730892181396, 0.0005673017585650086, 0.006459538359194994]}
1/1 [==============================] - 0s 61ms/step
{'argmax_index': 2, 'value_at_argmax': 0.9999968, 'pass_general_tollerance': True, 'class': 'in_flight', 'classes': ['char_select', 'connection_lost', 'in_flight', 'in_hanger'], 'scores': [1.6236298416849593e-10, 1.2359168593059633e-12, 0.9999967813491821, 3.2581651794316713e-06]}
saving...
1/1 [==============================] - 0s 47ms/step
{'argmax_index': 2, 'value_at_argmax': 1.0, 'pass_general_tollerance': True, 'class': 'jump_though_first', 'classes': ['dock_now', 'invalid', 'jump_though_first', 'jump_through_second', 'no_action', 'warp_to_dock_3', 'warp_to_dock_4'], 'scores': [6.55214001544363e-14, 1.4499611635052784e-14, 1.0, 3.99308651465382

KeyboardInterrupt: 

In [8]:
search_df = game.get_search_data(rows=13)
search_df = search_df[~search_df['Name'].str.contains('Jita')]
search_df

,Name,click_target
1,Itama Il = Perkone |,"(-994, 244)"
2,Itama VI ~ Moon 6,"(-994, 274)"
3,Itama VI ~ Perkone,"(-994, 304)"
4,Itama Vil ~ Maan 3,"(-994, 334)"
5,Itama Vil ~ Moan §,"(-994, 364)"
6,Itama Vil ~ Rapid A,"(-994, 394)"
7,Malkalen Iii ~ Ishukor,"(-994, 424)"
8,Malkalen 1V ~ Moon,"(-994, 454)"
9,Malkalen V ~ Moon 1,"(-994, 484)"
10,Veisto V - Caldari S,"(-994, 514)"


In [20]:
nav_point_xy = get_processed_cords(137, 378)
while True:
    pyautogui.moveTo(nav_point_xy)
    time.sleep(0.1)
    pyautogui.click(button='right')
    time.sleep(0.1)
    box = (102, 289, 458, 658)
    img = get_screen()
    state_result = execute_clsf(img, 'game_state')
    print(state_result)
    img = img.crop(box)
    nav_result = execute_clsf(img, 'nav_options')
    print(nav_result)
    if state_result['class'] == 'in_flight':
        if nav_result['class'] == 'dock_now':
            print('Docking Now...')
            pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 75)
            time.sleep(0.1)
            pyautogui.click(button='left')
        elif nav_result['class'] == 'jump_though_first':
            print('jumping fist pos...')
            pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 25)
            time.sleep(0.1)
            pyautogui.click(button='left')
        elif nav_result['class'] == 'jump_through_second':
            print('jumping second pos...')
            pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 50)
            time.sleep(0.1)
            pyautogui.click(button='left')
        elif nav_result['class'] == 'warp_to_dock':
            print('Warp To Dock pos...')
            pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 75)
            time.sleep(0.1)
            pyautogui.click(button='left')
        else:
            print('do nothing nav...')
    elif state_result['class'] == 'in_hanger':
        time.sleep(2)
        
        pyautogui.moveTo(get_processed_cords(*config['mining_hold_target']))  # ensure we have mining hold selected
        time.sleep(0.1)
        pyautogui.click(button='left')
        time.sleep(1)
        
        
        pyautogui.moveTo(get_processed_cords(*config['click_and_drag_inv_box'][2:4]))  #
        time.sleep(0.1)
        x, y = get_processed_cords(*config['click_and_drag_inv_box'][0:2])
        pyautogui.dragTo(x, y, 1, button='left')
        time.sleep(0.1)
        pyautogui.moveTo(get_processed_cords(*config['Unload_nav_bot_click_and_drag_inv_line'][0:2]))
        time.sleep(0.1)
        x, y = get_processed_cords(*config['Unload_nav_bot_click_and_drag_inv_line'][2:4])
        pyautogui.dragTo(x, y, 1, button='left')
        time.sleep(0.1)
        pyautogui.moveTo(get_processed_cords(*config['exit_hanger_target']))
        time.sleep(0.1)
        pyautogui.click(button='left')
    else:
        print('do nothing state...')
    time.sleep(0.1)
    pyautogui.moveTo(get_processed_cords(100, 100))
    time.sleep(0.1)
    pyautogui.click(button='left')
    time.sleep(10)

1/1 [==============================] - 0s 87ms/step
{'argmax_index': 2, 'value_at_argmax': 0.9999999, 'pass_general_tollerance': True, 'class': 'in_flight', 'classes': ['char_select', 'connection_lost', 'in_flight', 'in_hanger'], 'scores': [3.529085118754438e-13, 2.2406818357668078e-14, 0.9999998807907104, 1.2652982661620626e-07]}
1/1 [==============================] - 0s 47ms/step
{'argmax_index': 2, 'value_at_argmax': 1.0, 'pass_general_tollerance': True, 'class': 'jump_though_first', 'classes': ['dock_now', 'invalid', 'jump_though_first', 'jump_through_second', 'no_action', 'warp_to_dock'], 'scores': [1.0941613608861939e-16, 1.4693961325473737e-11, 1.0, 2.1681695580259418e-16, 1.1860082270863013e-15, 1.1774707490768647e-14]}
jumping fist pos...
1/1 [==============================] - 0s 24ms/step
{'argmax_index': 2, 'value_at_argmax': 0.99999976, 'pass_general_tollerance': True, 'class': 'in_flight', 'classes': ['char_select', 'connection_lost', 'in_flight', 'in_hanger'], 'scores': [

KeyboardInterrupt: 

In [ ]:
    img.save(f'O:\\source\\repos\\EVE-Online-Bot\\training_data\\unclass\\{uuid.uuid1()}.png')
    time.sleep(0.1)
    pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 25)
    time.sleep(0.1)
    pyautogui.click(button='left')
    time.sleep(0.1)
    pyautogui.moveTo(get_processed_cords(100, 100))
    time.sleep(0.1)
    pyautogui.click(button='left')
    time.sleep(10)

In [14]:
nav_point_xy = get_processed_cords(137, 378)

In [15]:
pyautogui.moveTo(nav_point_xy)
time.sleep(0.1)
pyautogui.click(button='right')
pyautogui.moveTo(get_processed_cords(10, 10))

In [37]:
box_top_left = (102, 289)
box_bottom_right = (458, 658)
img = get_screen()
img.crop([box_top_left[0], box_top_left[1], box_bottom_right[0], box_bottom_right[1]])
id = uuid.uuid1()
img.save(rf'O:\source\repos\EVE-Online-Bot\training_data\unclass\{}')

In [ ]:
time.sleep(0.1)
pyautogui.moveTo(nav_point_xy[0] + 50, nav_point_xy[1] + 25)
time.sleep(0.1)
pyautogui.click(button='left')
time.sleep(0.1)
pyautogui.moveTo(10, 10)